In [1]:
import csv
from datetime import datetime
import numpy as np
import pandas as pd
import time 
from datetime import datetime
import os
from os import path
from glob import glob

import ijson
from tqdm import tqdm

##  Connect hours of location data

In [3]:
def all_hours(area, filetype):
    # Path
    dfeb = "../results_location/hour"
    dmar = "../results_location2/hour"
    feb_files = glob(os.path.join(dfeb, 'area'+area+'_'+filetype+'*'))
    mar_files = glob(os.path.join(dmar, 'area'+area+'_'+filetype+'*'))

    # Get each csvs
    d_febs = [pd.read_csv(feb_file) for feb_file in feb_files]
    d_mars = [pd.read_csv(mar_file) for mar_file in mar_files]

    # Dates list
    dates_feb = [feb.split('/')[3].split('.')[0][7:] for feb in feb_files]
    dates_mar = [mar.split('/')[3].split('.')[0][7:] for mar in mar_files]

    # Add Day column
    # Feb
    for df, day in zip(d_febs, dates_feb):
        df.insert(0, 'day', day)
    # Mar
    for df, day in zip(d_mars, dates_mar):
        df.insert(0, 'day', day)

    # Concar Feb-data
    feb_hour = pd.concat(d_febs)
    # Concar Mar-data
    mar_hour = pd.concat(d_mars)

    # Concat all data
    A_hour =  pd.concat([feb_hour, mar_hour])
    
    # 処理
    A_hour[filetype] = A_hour.drop(['hour'], axis=1).sum(axis=1)
    cols = A_hour.columns.tolist()
    dropcols = cols[2:-1]
    A_hour = A_hour.drop(dropcols, axis=1)
    A_hour['area'] = area
    
    # Save to csv
    outname = 'area'+area+'_'+filetype + '.csv'
#     outname = 'test.csv'
    outdir = "../results_location" + '/hours'
    savepath = os.path.join(outdir, outname)
    if not os.path.exists(outdir):
            os.makedirs(outdir)
            
    # Save csv finally
    A_hour.to_csv(savepath, index=False)

In [4]:
# Run
areas = ['A','B','C','D','E','F','G','H','I','J']
for area in areas:
    filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
    for filetype in filetypes:
        test = all_hours(area, filetype)

## Summary

In [5]:
hour_files = glob('../results_location/hours/*')
dfs = [pd.read_csv(hour_file) for hour_file in hour_files]

In [6]:
for i, df in enumerate(dfs):
    if i == 0:
        df1 = df
    else:
        df1 = pd.concat([df1, df])
df1.groupby(['day', 'hour', 'area']).sum().to_csv('../results_location/summary_location_v2megi.csv')

## Test summary

In [7]:
dft = pd.read_csv('../results_location/summary_location_v2megi.csv')

In [15]:
dft[dft.area == 'A']

,day,hour,area,A,C,D,F,T,V
0,2020-02-01,0,A,1.0,0.0,0.0,0.0,1.0,0.0
16,2020-02-01,2,A,0.0,0.0,1.0,1.0,1.0,0.0
23,2020-02-01,3,A,1.0,0.0,0.0,0.0,0.0,0.0
31,2020-02-01,4,A,1.0,0.0,0.0,0.0,0.0,0.0
45,2020-02-01,6,A,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
12453,2020-03-31,17,A,0.0,1.0,1.0,1.0,1.0,0.0
12462,2020-03-31,18,A,1.0,0.0,0.0,1.0,1.0,0.0
12480,2020-03-31,20,A,0.0,1.0,0.0,1.0,1.0,1.0
12490,2020-03-31,21,A,1.0,0.0,1.0,0.0,1.0,0.0
